In [1]:
n_inputs = 3 #input vector size for each time stamp
n_neurons = 5

In [2]:
import tensorflow as tf
import numpy as np

In [20]:
# n_inputs = 3
# n_neurons = 5
# X0 = tf.placeholder(tf.float32, [None, n_inputs])
# X1 = tf.placeholder(tf.float32, [None, n_inputs])
# Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
# Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
# b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))
# Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
# Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)
# init = tf.global_variables_initializer()


In [56]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [57]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype = tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons]), dtype= tf.float32)

In [58]:
# tanh funciton plotted
# x = np.arange(-10, 10, 1)
# y = np.tanh(x)
# print(y)
# plt.plot(x,y)
Y0 = tf.tanh(tf.matmul(X0, Wx)+ b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) +  b)
init = tf.initializers.global_variables()

In [59]:
X0_batch = np.array([[0, 1, 2], 
                     [3, 4, 5], 
                     [6, 7, 8], 
                     [9, 0, 1]]) # at t=0
X1_batch = np.array([[9, 8, 7], 
                     [0, 0, 0],
                     [6, 5, 4],
                     [3, 2, 1]]) #at t=1


In [72]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict = { X0: X0_batch, X1 : X1_batch}) #fetches can be list, ordered Data. here [Y0, Y1] as fetches 
#     print(Y0_val, Y1_val)

In [76]:
print(Y0_val)
print(Y1_val)
# print(X0_batch.shape, np.shape(X0), X1_batch.shape, np.shape(X1))

[[ 0.9779369   0.86674     0.4873942   0.90718687 -0.9229775 ]
 [ 1.          0.96986324  0.03596565  1.         -0.99999684]
 [ 1.          0.99346393 -0.43063378  1.         -1.        ]
 [ 0.99999964 -1.         -0.9988381   0.9994498  -0.99877286]]
[[ 1.         -0.4508167  -0.8497606   1.         -1.        ]
 [ 0.9983719  -0.7717322   0.77549446  0.9670939  -0.70476377]
 [ 1.         -0.25396088 -0.31837085  1.         -0.9999998 ]
 [ 0.9999865  -0.95622706  0.13816315  0.9999336  -0.9974408 ]]


**USING TENSORFLOW FUNCTION**

In [78]:
# n_inputs = 3
# n_neurons = 5
# X0 = tf.placeholder(tf.float32, [None, n_inputs])
# X1 = tf.placeholder(tf.float32, [None, n_inputs])
# basicCell = tf.keras.layers.SimpleRNNCell(num_units = n_neurons, activation = tanh)
# # staticCell = tf.contrib.rnn.static_rnn(num_uni)
# states, outSeqs = tf.keras.layers.RNN(cell = basicCell)

In [1]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential

import numpy as np

Using TensorFlow backend.


In [10]:
openFile = open("Alice.txt", 'r')
alice = openFile.read()

In [11]:
from nltk.tokenize import sent_tokenize, WordPunctTokenizer
# alice_sent = sent_tokenize(alice)
# print(alice_sent[3:6])
alice_word_tokens = WordPunctTokenizer().tokenize(alice)
alice_stops = alice_word_tokens

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/bmrs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
# stopwords.fileids()
eng_stops = list(set(stopwords.words('english')))
# print(eng_stops)
alice_stops_free = [words for words in alice_stops if not words in eng_stops]
# alice_stops_free

In [ ]:
alice_stops_free

In [ ]:
alice_stops_final = print([' '.join(alice_stops_free)])

In [12]:
file_alice = open("Alice.txt", 'rb')
words = []
for word in file_alice:
    word = word.strip().lower()
    word = word.decode("ascii", "ignore")
    if len(word) == 0:
        continue
    words.append(word)
file_alice.close()
text = " ".join(words)

Creating Indexes to characters.
Make lookUp tables

In [13]:
chars = set([c for c in text])
len_chars = len(chars)
# list(enumerate(chars, start = 1))
index2char = dict(enumerate(chars))
char2index = dict((c,i) for i,c in enumerate(chars))
# char2index[]

In [60]:
f = open("Alice_clean.txt", "w+")
f.write(text)

158101

In [14]:
seqlen = 10
step = 1

In [16]:
input_chars = [] #contains seqlen: 10 characters
label_chars = []
for i in range(0, len(text)- seqlen, step):
    input_chars.append(text[i: i+seqlen])
    label_chars.append(text[i + seqlen])

**Vectorize the input and label texts**

In [17]:
X = np.zeros((len(input_chars), seqlen, len_chars), dtype= np.bool)
Y = np.zeros((len(input_chars), len_chars), dtype= np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    Y[i, char2index[label_chars[i]]] = 1
        

**BUILDING OUR MODEL**
- rnn OUTPUT DIMENSION SIZE = 128
- WE WANT TO RETURN A SINGLE CHARACTER AS OUTPUT, not a sequence of characters,then return_sequences =Fasle
- input to RNN i of shape: (seqlen, len_chars)
- set unroll = True, improoves performance of RNN

**Activation = Softmax, gives character with the highest pobability**

In [27]:
hidden_size = 128
batch_size = 128
num_iters = 35
num_epochs_per_iter = 1
num_preds_per_epoch = 100

In [28]:
#create model
model = Sequential()
model.add(SimpleRNN(units = 128, return_sequences = False, 
                    input_shape =(seqlen, len_chars), unroll = True))
model.add(Dense(len_chars))
model.add(Activation("softmax"))
#specify loss funcion
model.compile(loss = "categorical_crossentropy", optimizer = "rmsprop")

In [29]:
for i in range(num_iters):
    print("=" * 50)
    print("Iteration #: %d" %(i))
    model.fit(X, Y, batch_size = batch_size, epochs = num_epochs_per_iter)
    
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("generating from seed: %s" %(test_chars))
    print(test_chars, end="")
    for i in range(num_preds_per_epoch):
        Xtest = np.zeros((1, seqlen, len_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        #move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
print()

Iteration #: 0
Epoch 1/1
158091/158091 [==============================] - 14s 88us/step - loss: 2.3351
generating from seed: of hjckrrh
of hjckrrhe the said the guthe d at the gat she ghe she she she she she she she she she she she she she she sh==================================================
Iteration #: 1
Epoch 1/1
158091/158091 [==============================] - 14s 88us/step - loss: 2.0475
generating from seed: upon her: 
upon her: the was at in the was at in the was at in the was at in the was at in the was at in the was at in th==================================================
Iteration #: 2
Epoch 1/1
158091/158091 [==============================] - 15s 94us/step - loss: 1.9442
generating from seed: you mean, 
you mean, and the was suther alice and the was suther alice and the was suther alice and the was suther alice ==================================================
Iteration #: 3
Epoch 1/1
158091/158091 [==============================] - 14s 91us/step - loss: 1.8637
genera